In [1]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")
import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np

client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response



In [2]:
num_rows_batch_publish=5000 # number of objects to be batch published to Scorpio

# Notes:
Consider every hexagon as an entity, and sensor data as properties

In [3]:
import pandas as pd
import geopandas as gpd



In [4]:
# import building datasets
gdf_madrid = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-ESP_8_1_3_10_1-Madrid.gpkg')
gdf_berlin = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-DEU_3_1_1_1_1-Berlin.gpkg')
#gdf_warszawa = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-POL_7_36_1-Warszawa.gpkg')
#gdf_vichy = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-FRA_1_2_3_11_1-Vichy.gpkg')
#gdf_amsterdam = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-NLD_9_4_1-Amsterdam.gpkg')
#gdf_basel = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-CHE_5_1_1-Basel.gpkg')
#gdf_brugge = gpd.read_file('./../gsolmaz/data/eubucco-BuildingDataset/v0_1-BEL_2_5_1_3_1-Brugge.gpkg')

#gdf_amsterdam

In [5]:

madrid_hexagons_gdf = pd.read_pickle('./../cgarrido/madrid_data/hex_grid/hexagons.pkl')
berlin_hexagons_gdf=  pd.read_pickle('./../cgarrido/berlin_data/hex_grid/hexagons.pkl')
berlin_hexagons_gdf

,hexagon_id,hexagon_x,hexagon_y,geometry,lat,lon
0,0,0,0,"POLYGON ((1490671.563 6896601.483, 1490769.010...",52.525749,13.409063
1,1,1,1,"POLYGON ((1486732.502 6896205.302, 1486830.610...",52.523589,13.373677
2,2,0,1,"POLYGON ((1488848.594 6893262.031, 1488946.321...",52.507496,13.392682
3,3,1,0,"POLYGON ((1488555.212 6899545.594, 1488653.039...",52.541839,13.390056
4,4,-1,-1,"POLYGON ((1494611.363 6896995.788, 1494708.149...",52.527898,13.444455
...,...,...,...,...,...,...
266,266,-9,-7,"POLYGON ((1522486.772 6893415.233, 1522578.827...",52.508285,13.694856
267,267,-9,-9,"POLYGON ((1526156.027 6900082.526, 1526247.513...",52.544718,13.727827
268,268,-7,-9,"POLYGON ((1521933.780 6905992.816, 1522026.023...",52.577001,13.689907
269,269,-8,-9,"POLYGON ((1524045.347 6903037.871, 1524137.212...",52.560864,13.708871


In [6]:
madrid_hexagons_gdf

,hexagon_id,hexagon_x,hexagon_y,geometry,lat,lon
0,0,0,0,"POLYGON ((-409386.074 4929758.301, -410760.322...",40.424775,-3.674293
1,1,-1,-1,"POLYGON ((-406271.753 4930352.210, -407646.104...",40.428837,-3.646322
2,2,-1,0,"POLYGON ((-407280.695 4927212.167, -408654.636...",40.407365,-3.655384
3,3,0,-1,"POLYGON ((-408377.342 4932899.399, -409752.003...",40.446249,-3.665232
4,4,0,1,"POLYGON ((-410394.293 4926617.691, -411768.130...",40.403298,-3.683349
...,...,...,...,...,...,...
86,86,0,5,"POLYGON ((-414422.058 4914060.140, -415794.249...",40.317355,-3.719527
87,87,2,5,"POLYGON ((-418634.734 4919143.424, -420007.539...",40.352157,-3.757361
88,88,5,5,"POLYGON ((-424963.855 4926772.229, -426337.577...",40.404352,-3.814202
89,89,3,5,"POLYGON ((-420743.093 4921685.843, -422116.203...",40.369556,-3.776296


In [7]:


gdf_madrid= gdf_madrid.to_crs(epsg=3857)
gdf_berlin= gdf_berlin.to_crs(epsg=3857)

gdf_madrid_mapped_buildings = gpd.sjoin(gdf_madrid, madrid_hexagons_gdf).drop(columns=["index_right"])

gdf_berlin_mapped_buildings = gpd.sjoin(gdf_berlin, berlin_hexagons_gdf).drop(columns=["index_right"])

gdf_berlin_mapped_buildings


,id,height,age,type,id_source,type_source,geometry,hexagon_id,hexagon_x,hexagon_y,lat,lon
0,v0.1-DEU.3.1.1.1_1-0,2.77,NaN,NaN,BLDG_00030000006a7af7,2862.0,"POLYGON ((1510943.155 6878374.869, 1510943.157...",143,-7,-1,52.431146,13.558297
1,v0.1-DEU.3.1.1.1_1-1,2.38,NaN,NaN,BLDG_00030000006a7ac7,2862.0,"POLYGON ((1510930.698 6878361.523, 1510930.696...",143,-7,-1,52.431146,13.558297
2,v0.1-DEU.3.1.1.1_1-2,5.43,NaN,NaN,BLDG_0003000f00190176,1301.0,"POLYGON ((1510930.690 6878423.168, 1510930.689...",143,-7,-1,52.431146,13.558297
3,v0.1-DEU.3.1.1.1_1-3,4.27,NaN,NaN,BLDG_000300090023bdbd,1004.0,"POLYGON ((1510934.982 6878223.712, 1510934.981...",143,-7,-1,52.431146,13.558297
4,v0.1-DEU.3.1.1.1_1-4,3.31,NaN,NaN,BLDG_0003000f00190236,1301.0,"POLYGON ((1510813.713 6878448.665, 1510813.711...",143,-7,-1,52.431146,13.558297
...,...,...,...,...,...,...,...,...,...,...,...,...
538438,v0.1-DEU.3.1.1.1_1-538438,0.93,NaN,NaN,DEB_LOD2_UUID_2b05c943-0a28-4567-89c9-e652996d...,NaN,"POLYGON ((1485916.101 6910904.824, 1485916.100...",70,4,-1,52.608301,13.349368
538449,v0.1-DEU.3.1.1.1_1-538449,1.54,NaN,NaN,DEB_LOD2_UUID_c1ef21de-50ca-4d37-8118-33e4f456...,NaN,"POLYGON ((1486106.968 6910788.338, 1486106.966...",70,4,-1,52.608301,13.349368
538454,v0.1-DEU.3.1.1.1_1-538454,2.49,NaN,NaN,DEB_LOD2_UUID_c2fe762c-e122-40d9-9866-7102e0df...,NaN,"POLYGON ((1485910.027 6910886.942, 1485910.027...",70,4,-1,52.608301,13.349368
538460,v0.1-DEU.3.1.1.1_1-538460,2.88,NaN,NaN,DEB_LOD2_UUID_f73dce9d-8b92-4b65-89ee-ede0a978...,NaN,"POLYGON ((1484739.903 6909988.569, 1484739.903...",70,4,-1,52.608301,13.349368


In [8]:
import geojson    
import pandas as pd
from geojson import Polygon
from pyproj import Transformer





transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326") 

def transform_polygon(hexagon_geometry):
    
    xx, yy= hexagon_geometry.exterior.coords.xy
    
    points=[]
    for i in range(len(xx)): # for all points x,y
        result = transformer.transform( xx[i] , yy[i])
        result = (result[1], result[0])
        points.append(result)
    polygon = Polygon([points])  
    return polygon



In [9]:

gdf_berlin_mapped_buildings['geometry'] = gdf_berlin_mapped_buildings['geometry'].apply(transform_polygon)
gdf_madrid_mapped_buildings['geometry'] = gdf_madrid_mapped_buildings['geometry'].apply(transform_polygon)

gdf_berlin_mapped_buildings

,id,height,age,type,id_source,type_source,geometry,hexagon_id,hexagon_x,hexagon_y,lat,lon
0,v0.1-DEU.3.1.1.1_1-0,2.77,NaN,NaN,BLDG_00030000006a7af7,2862.0,"POLYGON ((13.573 52.431, 13.573 52.431, 13.573...",143,-7,-1,52.431146,13.558297
1,v0.1-DEU.3.1.1.1_1-1,2.38,NaN,NaN,BLDG_00030000006a7ac7,2862.0,"POLYGON ((13.573 52.431, 13.573 52.431, 13.573...",143,-7,-1,52.431146,13.558297
2,v0.1-DEU.3.1.1.1_1-2,5.43,NaN,NaN,BLDG_0003000f00190176,1301.0,"POLYGON ((13.573 52.431, 13.573 52.431, 13.573...",143,-7,-1,52.431146,13.558297
3,v0.1-DEU.3.1.1.1_1-3,4.27,NaN,NaN,BLDG_000300090023bdbd,1004.0,"POLYGON ((13.573 52.430, 13.573 52.430, 13.573...",143,-7,-1,52.431146,13.558297
4,v0.1-DEU.3.1.1.1_1-4,3.31,NaN,NaN,BLDG_0003000f00190236,1301.0,"POLYGON ((13.572 52.431, 13.572 52.431, 13.572...",143,-7,-1,52.431146,13.558297
...,...,...,...,...,...,...,...,...,...,...,...,...
538438,v0.1-DEU.3.1.1.1_1-538438,0.93,NaN,NaN,DEB_LOD2_UUID_2b05c943-0a28-4567-89c9-e652996d...,NaN,"POLYGON ((13.348 52.608, 13.348 52.608, 13.348...",70,4,-1,52.608301,13.349368
538449,v0.1-DEU.3.1.1.1_1-538449,1.54,NaN,NaN,DEB_LOD2_UUID_c1ef21de-50ca-4d37-8118-33e4f456...,NaN,"POLYGON ((13.350 52.608, 13.350 52.608, 13.350...",70,4,-1,52.608301,13.349368
538454,v0.1-DEU.3.1.1.1_1-538454,2.49,NaN,NaN,DEB_LOD2_UUID_c2fe762c-e122-40d9-9866-7102e0df...,NaN,"POLYGON ((13.348 52.608, 13.348 52.608, 13.348...",70,4,-1,52.608301,13.349368
538460,v0.1-DEU.3.1.1.1_1-538460,2.88,NaN,NaN,DEB_LOD2_UUID_f73dce9d-8b92-4b65-89ee-ede0a978...,NaN,"POLYGON ((13.338 52.603, 13.338 52.603, 13.338...",70,4,-1,52.608301,13.349368


In [10]:
gdf_list= [gdf_madrid_mapped_buildings, gdf_berlin_mapped_buildings]
entity_id_list = [ "urn:ngsi-ld:Building:Madrid", "urn:ngsi-ld:Building:Berlin"]
entity_type_list = ['Building'] * len(gdf_list)


,id,height,age,type,id_source,type_source,geometry,hexagon_id,hexagon_x,hexagon_y,lat,lon


In [64]:
### Create Hexagon entites as aggregated building information 
### Create building entities for each building
    ### Add relation from hexagon to the building
import uuid
from geojson.geometry import Point, Polygon
import shapely
import time
import json
import math

example_flag=False
example_hexagon_flag= False
for entity_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
    ingestion_starttime= time.perf_counter() 
    example_flag=True
    example_hexagon_flag= True

    hexagon_id_list = tmp_gdf.hexagon_id.unique()
    print("Info: Unique hexagons for the dataset : "+ str(hexagon_id_list))
    feature_list= tmp_gdf.columns.tolist()
    dataset_length = len(tmp_gdf)
    for hexagon_id in hexagon_id_list:
        #print(entity_id_list[entity_index] + ':Hexagon:' + str(hexagon_id))
        hexagon_entity_obj = Entity(entity_type_list[entity_index], entity_id_list[entity_index] + ':Hexagon:' + str(hexagon_id)) # (re-)initialize the entity
        hexagon_num_buildings=0
        hexagon_height_list=[]
        hexagon_gdf = tmp_gdf[tmp_gdf['hexagon_id']== hexagon_id]

        counter =0
        for index, row in hexagon_gdf.iterrows():
            building_entity_id = entity_id_list[entity_index] + ':BuildingID:'  + str(row.id)
            building_entity_obj=  Entity(entity_type_list[entity_index], building_entity_id)
            for f in range(len(feature_list)):
                if feature_list[f] =='type': feature_list[f] ='building_type' # quick fix for the error of "type" conflicting with ngsi-ld "type"
                if(feature_list[f] not in ['datetime','type', 'geometry']):
                    if(isinstance(row[f], str) or not math.isnan(float(row[f]))):
                        building_entity_obj.add_prop(feature_list[f], row[f]) 
                    else: # the value is nan, add as -1
                        building_entity_obj.add_prop(feature_list[f], -1)
                elif(feature_list[f] == 'geometry'):
                    s= geojson.dumps(row[f])
                    data = json.loads(s)
                    h = Polygon(data['coordinates'])  
                    building_entity_obj.add_geoprop("location", h)
                if feature_list[f] == 'hexagon_id':
                    tmp_uuid = uuid.uuid4()
                    hexagon_entity_obj.add_rel(name= 'providedBy', object= building_entity_id, multi=True, datasetId= str(tmp_uuid))
                hexagon_num_buildings+=1
                hexagon_height_list.append(row.height)
                    
            if example_flag:
                print('INFO: Example building data from the dataset to be published - as an example on how data looks.')
                print(building_entity_obj)
                example_flag=False

            if (counter %num_rows_batch_publish==0 or counter == len(hexagon_gdf)-1) and  counter!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
                publish(building_entity_obj.to_dict()) # publish the entity
            counter+=1 
            
        hexagon_entity_obj.add_prop('number_of_buildings', hexagon_num_buildings)
        hexagon_height_list= np.asarray(hexagon_height_list)
    
        hexagon_entity_obj.add_prop('average_height_of_buildings', np.nanmean(hexagon_height_list))
        publish(hexagon_entity_obj.to_dict()) 
        print("... published data for hexagon: " +  str(hexagon_id))
        if(example_hexagon_flag):
            print('Published hexagon data')
            print(hexagon_entity_obj)
            example_hexagon_flag=False

    ingestion_endtime= time.perf_counter() 
    print('Info: Dataset is published successfully with entity ID: ' + str(entity_id_list[entity_index]) 
          + ' --- It took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds to publish the dataset with length ' + str(dataset_length))


Info: Unique hexagons for the dataset : [56 57 73 50 60 58 11 18 77 62 26 63 30 78 13 52 44 81 43 66 80 46  6 16
 67 24  5 39 53 37 21 65 41 48 42 22 69 20 23 61 14  4 71 40 70 38 64 28
  0 25 79 75 12 49  3 90 76 84 45 47 89 55 15  2 29  1  7 10 87 59  9 34
 36 31 35 17 19  8 32 54 27 33 85 83 68]
INFO: Example building data from the dataset to be published - as an example on how data looks.
{'id': {'type': 'Property', 'value': 'v0.1-ESP.8.1.3.10_1-111'}, 'type': 'Building', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'height': {'type': 'Property', 'value': 2.5}, 'age': {'type': 'Property', 'value': 2000.0}, 'building_type': {'type': 'Property', 'value': 'non-residential'}, 'id_source': {'type': 'Property', 'value': 'ES.SDGC.BU.000900200VK36F'}, 'type_source': {'type': 'Property', 'value': '3_industrial'}, 'location': {'type': 'GeoProperty', 'value': {"coordinates": [[[-3.72379, 40.344845], [-3.723765, 40.344963], [-3.723703, 40.344956], [-3.723728, 40

/tmp/ipykernel_44391/1806797390.py:63: RuntimeWarning: Mean of empty slice
  hexagon_entity_obj.add_prop('average_height_of_buildings', np.nanmean(hexagon_height_list))


... published data for hexagon: 28
... published data for hexagon: 0
... published data for hexagon: 25
... published data for hexagon: 79
... published data for hexagon: 75
... published data for hexagon: 12
... published data for hexagon: 49
... published data for hexagon: 3
... published data for hexagon: 90
... published data for hexagon: 76
... published data for hexagon: 84
... published data for hexagon: 45
... published data for hexagon: 47
... published data for hexagon: 89
... published data for hexagon: 55
... published data for hexagon: 15
... published data for hexagon: 2
... published data for hexagon: 29
... published data for hexagon: 1
... published data for hexagon: 7
... published data for hexagon: 10
... published data for hexagon: 87
... published data for hexagon: 59
... published data for hexagon: 9
... published data for hexagon: 34
... published data for hexagon: 36
... published data for hexagon: 31
... published data for hexagon: 35
... published data for hex